## Learning pytorch by training using MNIST

Second script we train feed forward networks and fully connected operator. We start moving from theano based code 
to Keras based code.

## Things observed in this code:

* How can we optmize the parameters easily.

* how can we use the torchvision interface to load a provided dataset.

* what is torch.no_grad() and how to use

In [6]:
import torch #main module
if not torch.cuda.is_available():
	print("Buy a gpu")
	exit(-1)
import torchvision #computer vision dataset module
from torchvision import datasets,transforms
from torch import nn #not use in this tutorials. Perfect for weepers and for Keras Users.

import numpy

Buy a gpu


C:\Users\jazfe\anaconda3\envs\deep_learning\lib\site-packages\torchvision\io\image.py:14: UserWarning: Failed to load image Python extension: '[WinError 1920] El sistema no tiene acceso al archivo'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [10]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
import numpy 
from torchvision import datasets, transforms

# se importan las librerías necesarias, incluyendo PyTorch, numpy y torchvision.

In [12]:
# Transformaciones y carga de datos
mnist_transforms = transforms.Compose([transforms.ToTensor()])
mnist_train = datasets.MNIST('/tmp/', train=True, download=True, transform=mnist_transforms)
mnist_test = datasets.MNIST('/tmp/', train=False, download=False, transform=mnist_transforms)

# Se definen las transformaciones (convertir imágenes a tensores) y se descargan/cargan los conjuntos de datos MNIST.

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /tmp/MNIST\raw\train-images-idx3-ubyte.gz to /tmp/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /tmp/MNIST\raw\train-labels-idx1-ubyte.gz to /tmp/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting /tmp/MNIST\raw\t10k-images-idx3-ubyte.gz to /tmp/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting /tmp/MNIST\raw\t10k-labels-idx1-ubyte.gz to /tmp/MNIST\raw



In [15]:
# DataLoader
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=True)

# Se crean los cargadores de datos (DataLoader) para el entrenamiento y prueba, 
# que permiten manejar los datos en lotes de 100 y barajarlos en cada época.

In [17]:
# Parámetros
l1, l2, l3 = 512, 512, 10
input_d, target_d, batch, epochs = 784, 10, 100, 50

# Se establecen los parámetros para las dimensiones de las capas y el entrenamiento, 
# como el tamaño del lote y el número de épocas.

In [19]:
# Inicialización de pesos y bias
w1 = torch.from_numpy(numpy.random.normal(0, numpy.sqrt(2.0/float(l1)), (input_d, l1)).astype('float32')).requires_grad_()
w2 = torch.from_numpy(numpy.random.normal(0, numpy.sqrt(2.0/float(l2)), (l1, l2)).astype('float32')).requires_grad_()
w3 = torch.from_numpy(numpy.random.normal(0, numpy.sqrt(2.0/float(l2)), (l2, l3)).astype('float32')).requires_grad_()

b1 = torch.from_numpy(numpy.zeros((1, l1))).float().requires_grad_()
b2 = torch.from_numpy(numpy.zeros((1, l2))).float().requires_grad_()
b3 = torch.from_numpy(numpy.zeros((1, l3))).float().requires_grad_()

fRl = nn.ReLU()

# Se inicializan los pesos y sesgos del modelo con valores aleatorios y se establece que requieren gradientes para el entrenamiento.

# Se define la función de activación ReLU y la función de pérdida de entropía cruzada (abajo, CrossEntropyLoss), 
# que se utilizarán durante el entrenamiento.

In [21]:
# Optimización
optimized_params = [b1, b2, b3, w1, w2, w3]
Loss = nn.CrossEntropyLoss()
optimizer = optim.SGD(optimized_params, lr=0.1, momentum=0.9)

# Se configuran los parámetros a optimizar y se utiliza el optimizador SGD con una tasa de aprendizaje y momento especificados.

In [23]:
# Entrenamiento
for e in range(epochs):
    ce, MC = 0.0, 0.0
    for x, t in train_loader:  # sample one batch
        x, t = x.view(-1, 784), t
        predict = torch.mm(fRl(torch.mm(fRl(torch.mm(x, w1) + b1), w2) + b2), w3) + b3  # forward
        o = Loss(predict, t)  # compute loss
        o.backward()  # compute gradients

        optimizer.step()
        optimizer.zero_grad()
        ce += o.item()

    with torch.no_grad():
        for x, t in test_loader:
            x, t = x.view(-1, 784), t
            predict = torch.mm(fRl(torch.mm(fRl(torch.mm(x, w1) + b1), w2) + b2), w3) + b3  # forward
            index = torch.argmax(predict, 1)  # compute maximum
            MC += ((index != t).float().sum()).item()  # accumulate error

    print(f"|| Epoch {e} cross entropy {ce/600:.5f} and Test error {100*MC/10000:.3f}")

# Aquí se entrena el modelo durante el número de épocas definido. En cada iteración, se hace una pasada hacia adelante, 
# se calcula la pérdida, se realiza la retropropagación para calcular los gradientes y se actualizan los parámetros.

# Finalmente, el modelo se evalúa en el conjunto de prueba al final de cada época para calcular el error y la precisión.

|| Epoch 0 cross entropy 0.20941 and Test error 3.070
|| Epoch 1 cross entropy 0.08379 and Test error 3.200
|| Epoch 2 cross entropy 0.05468 and Test error 2.640
|| Epoch 3 cross entropy 0.03836 and Test error 2.190
|| Epoch 4 cross entropy 0.02688 and Test error 2.010
|| Epoch 5 cross entropy 0.01847 and Test error 1.860
|| Epoch 6 cross entropy 0.01177 and Test error 1.710
|| Epoch 7 cross entropy 0.01321 and Test error 1.970
|| Epoch 8 cross entropy 0.01197 and Test error 1.770
|| Epoch 9 cross entropy 0.00426 and Test error 1.880
|| Epoch 10 cross entropy 0.00411 and Test error 2.020
|| Epoch 11 cross entropy 0.00538 and Test error 1.780
|| Epoch 12 cross entropy 0.00437 and Test error 1.770
|| Epoch 13 cross entropy 0.00453 and Test error 1.630
|| Epoch 14 cross entropy 0.00226 and Test error 1.610
|| Epoch 15 cross entropy 0.00082 and Test error 1.460
|| Epoch 16 cross entropy 0.00014 and Test error 1.410
|| Epoch 17 cross entropy 0.00007 and Test error 1.400
|| Epoch 18 cross en

* Epoch: Esta es la primera época de entrenamiento. Una época significa que el modelo ha pasado por todo el conjunto de datos de entrenamiento una vez.

* Cross Entropy: 0.20941 es el valor de la pérdida de entropía cruzada después de la primera época. La entropía cruzada es una medida de la diferencia entre las predicciones del modelo y las etiquetas reales. Cuanto más bajo sea este número, mejor es el rendimiento del modelo en el conjunto de datos de entrenamiento.

* Test Error: 3.070% es el error en el conjunto de prueba después de la primera época. Esto indica el porcentaje de predicciones incorrectas en el conjunto de datos de prueba.

Este código entrena una red neuronal simple en el conjunto de datos MNIST. Pasa los datos a través de una red de tres capas, calcula la pérdida de entropía cruzada y ajusta los pesos usando el optimizador SGD. Después de cada época, el modelo se evalúa en el conjunto de datos de prueba para medir su rendimiento.

Un valor más bajo de *entropía cruzada* indica que el modelo está haciendo predicciones más precisas en el conjunto de entrenamiento.
El error en el *conjunto de prueba* también ha disminuido, lo que sugiere que el modelo está generalizando bien a los datos no vistos.

### Análisis de resultados: 

Durante el entrenamiento del modelo de red neuronal en el conjunto de datos MNIST, se observaron resultados positivos a lo largo de 50 épocas. La pérdida de entropía cruzada, que mide la diferencia entre las predicciones y las etiquetas reales, disminuyó de 0.20941 en la época 0 a 0.00001 en la época 49. Esto indica que el modelo está aprendiendo eficazmente de los datos de entrenamiento.

El error en el conjunto de prueba también mostró una disminución, pasando del 3.070% en la época 0 al 1.410% en la época 49. A pesar de ligeras fluctuaciones, el error se estabilizó alrededor del 1.4%, lo que sugiere que el modelo generaliza bien y mantiene una capacidad de predicción estable en datos no vistos.

En resumen, el modelo está funcionando correctamente, aprendiendo eficazmente de los datos de entrenamiento y manteniendo un buen rendimiento en el conjunto de prueba.